## A Simulated IOT Federated Learning Framework for Forest Fire Prediction

Forest fire ignition prediction is essential to the safety of communities and the proper allocation of fire fighting resources. This notebook simulates a network of clients that collect local meteorological data. Each client participates in a federated learning system where a client trains a machine learning model to classify whether the area will have a forest fire ignition. The model weights of each client are shared with a central server that averages the weights and shares them back to the clients.

This notebook demonstrates the two machine learning types shown below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>central server ML|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated ML (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>federated|<center>on clients|<center>on clients|


The data used is from the paper: *Framework for Creating Forest Fire Ignition Prediction Datasets.* Each row represents meteorological data at a geographical location at a specific time. TODO: Add table example.

Much of the code used in this notebook is based on the Flower code examples located [here](https://github.com/adap/flower/tree/main/examples) and the Keras timeseries tutorials located [here](https://keras.io/examples/timeseries/).
The code below is very much a work in progress.

In [1]:
#if this file is being used in colab set to 1 otherwise 0
using_colab = 1

In [2]:
if (using_colab == 1):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if (using_colab ==1):
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
#load libraries
import math
import os
import glob
import gc
import datetime
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile
from tables.file import File

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

In [5]:
if (using_colab == 1):
    !pip install -q flwr[simulation] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.9

In [6]:
import flwr as fl
from flwr.common import Metrics

In [7]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
#tf.config.set_visible_devices([], 'CPU')
python_version = !python --version
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'}\
    using TensorFlow {tf.__version__}, Flower {fl.__version__} and {python_version[0]}"
)

Training on CPU    using TensorFlow 2.12.0, Flower 1.4.0 and Python 3.10.12


In [8]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

cid = str(0) # preliminary client id

master_path = "/content/drive/MyDrive/Colab Notebooks/FF/"
federated_path = master_path + "data/24_clients/"
centralized_path = master_path + "data/01_clients/"
results_path = master_path + "history/"
temp_path = master_path + "history_temp/"
log_dir = master_path + "tensorflow_logs/" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#master_path = <path to your master directory for this simulation>
#federated_path = <path to your client datasets>
#centralized_path = <path to your server dataset
#results_path = <path to where you want to store results>
#log_dir = <path to where you want to store Tensorflow logs> + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
NUM_CLIENTS = 24
num_rounds = 3
epochs = 10 # the number of epochs classic machine learning will use
fl_epochs = 3

sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart

FRACTION_FIT=0.75,  # Sample X% of available clients for training
FRACTION_EVALUATE=0.8,  # Sample X% of available clients for evaluation
MIN_FIT_CLIENTS=13,  # Never sample less than X clients for training
MIN_EVALUATE_CLIENTS=12,  # Never sample less than X clients for evaluation
MIN_AVAILABLE_CLIENTS=13,  # Wait until at least X clients are available

#create an array for final results
np.set_printoptions(suppress=True)
results = np.empty([15,2])

#create an array for interim results
metric_names = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'sensitivity', 'specificity']
metric_names_len = len(metric_names)
interim_results = np.zeros((NUM_CLIENTS, metric_names_len, num_rounds))

## Data loading functions

In [9]:
def normalize_data(x):
    """Normalizes the data of an array by column.
    Shifts and scales inputs into a distribution centered around 0
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray
        An array of feature values.

    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [10]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of
    negative labels = the number of positive labels.

    Parameters
    ----------
    x: NDarray
        An array of feature values.

    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0
    while count < ignition_count:
        #rand_num = random.randint(0,mask_length)
        rand_num = random.randint(1, mask_length-1)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [11]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string
        The path to the dataset folder.

    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])

        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]

        features = ['stl2',
                    't2m',
                    'stl1',
                    'stl3',
                    'skt',
                    'swvl1',
                    'd2m',
                    'swvl2'
                    ] #this shoudn't be hard coded
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)

        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values

        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break

        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)

        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)

        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)

        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)

    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y



In [12]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray
        An array with class labels.

    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count


## Model and Metrics

In [13]:

METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    keras.metrics.SensitivityAtSpecificity(0.5, name='sensitivity'),
    keras.metrics.SpecificityAtSensitivity(0.5, name='specificity')
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)

    return model


## Federated Learning Functions

In [14]:
class FlowerClient(fl.client.NumPyClient):
    """Instantiates a FlowerClient object

    Parameters
    ----------
    fl.client: NumPyClient
        A premade helper client.
    """
    def __init__(self, cid, model, x_train, y_train, x_val, y_val, x_test, y_test, tb_callback) -> None:
        self.cid = cid
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.x_test, self.y_test = x_test, y_test
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        server_round = config["server_round"]
        print("in server round: ", server_round)
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback, validation_data=(self.x_val, self.y_val),)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        server_round = int(config["server_round"])
        print(f"Hi, this is base station {self.cid} in eval for round {server_round}")
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc, sensitivity, specificity  = self.model.evaluate(self.x_test, self.y_test, verbose=2)



        # TODO: store the client round data somewhere internally
        # This section writes the client data to a file.
        # It is temporary until I can write a custom client function.

        client_round_results = np.zeros(13)

        print("\nWriting out interim results.")
        #write to interim results
        client_round_results[0] = self.cid
        client_round_results[1] = server_round
        client_round_results[2] = tp
        client_round_results[3] = fp
        client_round_results[4] = tn
        client_round_results[5] = fn
        client_round_results[6] = accuracy
        client_round_results[7] = precision
        client_round_results[8] = recall
        client_round_results[9] = auc
        client_round_results[10] = prc
        client_round_results[11] = sensitivity
        client_round_results[12] = specificity

        file = temp_path + str(self.cid) + "_" + str(server_round) + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
        pd.DataFrame(client_round_results).to_csv(file)

        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc,
                                       "sensitivity": sensitivity,
                                       "specificity": specificity
                                      }

    # this method allows metrics to be passed when using a single server with no clients
    def evaluate_single(self, x_test, y_test):
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc, sensitivity, specificity = self.model.evaluate(x_test, y_test, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc, sensitivity, specificity

In [15]:
def client_fn(cid: str) -> fl.client.Client:
    """Instantiates the client object.

    Parameters
    ----------
    cid: str
        The client id.

    Returns
    -------
    cid : NDarray
    model: model
    test/val/test datasets: NDarray
    tensorboard: tensorflow callbacks
        Everything a client needs to create a ML model
    """

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_val_cid = valloaders_x[int(cid)]
    y_val_cid = valloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)

        return model

    print("Making model: ", cid)
    model = make_model()

    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(cid, model, x_train_cid, y_train_cid, x_val_cid, y_val_cid, x_test_cid, y_test_cid, tensorboard)

In [16]:
# TODO: this function isn't working correctly, have a look at it
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """A function for the final evaluation of the simulation.
    All metrics are averaged and output.

    Parameters
    ----------
    metrics: List
        A list of the metrics being used

    Returns
    -------
     : Dict
        A dictionary of the averaged results from each round.
    """
    # Multiply each metric of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    sensitivitys = [num_examples * m["sensitivity"] for num_examples, m in metrics]
    specificitys = [num_examples * m["specificity"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples),
            "sensitivity": sum(sensitivitys)/ sum(examples),
            "specificity": sum(specificitys)/ sum(examples)
           }

In [17]:
def fit_config(server_round: int):
    """Passes information from the server to client.

    Parameters
    ----------
    server_round: int
        What server round simulation is in.

    Returns
    -------
    config : Dict
        A dictionary of values.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        #"local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

## Launch classic machine learning

In [18]:
# load the dataset for centralized evaluation (for classic ml training and testing)
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [19]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)


In [20]:
# create an initial model to prime flower client
model = make_model()

In [21]:
# train single server classic ml
# create a flower client that represents a classic ml single server
central_server_model = client_fn(str(0))
# set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
# run fit
config = fit_config(0)
history = central_server_model.fit(parameters, config)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
in server round:  0
Epoch 1/10
176/176 - 14s - loss: 0.6600 - tp: 1520.0000 - fp: 875.0000 - tn: 1936.0000 - fn: 1291.0000 - accuracy: 0.6147 - precision: 0.6347 - recall: 0.5407 - auc: 0.6563 - prc: 0.6513 - sensitivity: 0.6773 - specificity: 0.7076 - val_loss: 0.6215 - val_tp: 1826.0000 - val_fp: 899.0000 - val_tn: 1635.0000 - val_fn: 708.0000 - val_accuracy: 0.6829 - val_precision: 0.6701 - val_recall: 0.7206 - val_auc: 0.7234 - val_prc: 0.6931 - val_sensitivity: 0.8157 - val_specificity: 0.8070 - 14s/epoch - 82ms/step
Epoch 2/10
176/176 - 9s - loss: 0.6245 - tp: 1910.0000 - fp: 1016.0000 - tn: 1795.0000 - fn: 901.0000 - accuracy: 0.6590 - precision: 0.6528 - recall: 0.6795 - auc: 0.7152 - prc: 0.7154 - sensitivity: 0.7723 - specificity: 0.8051 - val_loss: 0.6106 - val_tp: 1800.0000 - val_fp: 864.0000 - val_tn: 1670.0000 - val_fn: 734.0000 - val_accuracy: 0.6847 - val_precision: 0.6757 

In [22]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc, sensitivity, specificity = central_server_model.evaluate_single(testloaders_x[0], testloaders_y[0])

1440/1440 - 15s - loss: 0.5485 - tp: 885.0000 - fp: 11002.0000 - tn: 33717.0000 - fn: 476.0000 - accuracy: 0.7509 - precision: 0.0745 - recall: 0.6503 - auc: 0.7637 - prc: 0.0878 - sensitivity: 0.8611 - specificity: 0.8438 - 15s/epoch - 10ms/step


In [23]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\n\
\tsens:\t%f\n\
\tspec:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc,sensitivity,specificity))

Single server classic ml evaluation
	tp:	885
	fp:	11002
	tn:	33717
	fn:	476

	loss:	0.548463
	acc:	0.750911
	prec:	0.074451
	rec:	0.650257
	auc:	0.763666
	prc:	0.087752
	sens:	0.861132
	spec:	0.843802


In [24]:
#write to results array
results[0] = tp
results[1][0] = fp
results[2][0] = tn
results[3][0] = fn
results[4][0] = accuracy
results[5][0] = precision
results[6][0] = recall
results[7][0] = auc
results[8][0] = prc
results[9][0] = sensitivity
results[10][0] = specificity
results[11][0] = epochs
results[12][0] = 1
results[13][0] = 1
results[14][0] = 1

In [25]:
#remove data from memory that isn't needed
for_removal = [trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y]
del(for_removal)
del(model)
del(central_server_model)
gc.collect()
print('Central server data removed.')

Central server data removed.


## Federated machine learning

The Flower federated learning framework is used from here down. More info can be found at https://flower.dev

In [26]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/121.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/120.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (460

In [27]:
#change the number of epochs from single server to federated
epochs = fl_epochs

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT[0],  # Sample 10% of available clients for training
    fraction_evaluate=FRACTION_EVALUATE[0],  # Sample 5% of available clients for evaluation
    min_fit_clients=MIN_FIT_CLIENTS[0],  # Never sample less than 10 clients for training
    min_evaluate_clients=MIN_EVALUATE_CLIENTS[0],  # Never sample less than 5 clients for evaluation
    min_available_clients=MIN_AVAILABLE_CLIENTS[0],  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    on_fit_config_fn=fit_config, # Pass the fit_config function
    on_evaluate_config_fn=fit_config # Pass the eval_config function
)

# Start simulation
fl_history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
)


INFO flwr 2023-06-13 17:21:54,287 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
2023-06-13 17:21:57,527	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-13 17:21:58,813 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 16241036083.0, 'memory': 32482072167.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 16241036083.0, 'memory': 32482072167.0}
INFO flwr 2023-06-13 17:21:58,818 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-13 17:21:58,823 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-06-13 17:22:05,020 | server.py:277 | Receiv

(launch_and_get_parameters pid=9811) 
(launch_and_get_parameters pid=9811) This is client:  17
(launch_and_get_parameters pid=9811) Loaded data for client:  17 
(launch_and_get_parameters pid=9811) 
(launch_and_get_parameters pid=9811) Making model:  17
(launch_and_get_parameters pid=9811) 
(launch_and_get_parameters pid=9811) Client CID: 17 is done.
(launch_and_get_parameters pid=9811) 
(launch_and_fit pid=9811) 
(launch_and_fit pid=9811) This is client:  5
(launch_and_fit pid=9811) Loaded data for client:  5 
(launch_and_fit pid=9811) 
(launch_and_fit pid=9811) Making model:  5
(launch_and_fit pid=9811) 
(launch_and_fit pid=9811) Client CID: 5 is done.
(launch_and_fit pid=9811) 
(launch_and_fit pid=9811) in fit
(launch_and_fit pid=9811) in server round:  1
(launch_and_fit pid=9811) Epoch 1/2
(launch_and_fit pid=9811) 7/7 - 10s - loss: 0.7043 - tp: 885.0000 - fp: 11004.0000 - tn: 33812.0000 - fn: 573.0000 - accuracy: 0.7498 - precision: 0.0744 - recall: 0.6070 - auc: 0.7598 - prc: 0.0

DEBUG flwr 2023-06-13 17:27:41,065 | server.py:232 | fit_round 1 received 18 results and 0 failures
DEBUG:flwr:fit_round 1 received 18 results and 0 failures
WARNING flwr 2023-06-13 17:27:41,120 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-06-13 17:27:41,125 | server.py:168 | evaluate_round 1: strategy sampled 19 clients (out of 24)
DEBUG:flwr:evaluate_round 1: strategy sampled 19 clients (out of 24)


(launch_and_evaluate pid=9811) This is client:  10
(launch_and_evaluate pid=9811) Loaded data for client:  10 
(launch_and_evaluate pid=9811) Making model:  10
(launch_and_fit pid=9805) Making model:  10
(launch_and_evaluate pid=9811)  [repeated 2x across cluster]
(launch_and_evaluate pid=9805) Client CID: 11 is done.
(launch_and_evaluate pid=9805) Hi, this is base station 11 in eval for round 1
(launch_and_evaluate pid=9805) 1440/1440 - 21s - loss: 0.6396 - tp: 1127.0000 - fp: 29432.0000 - tn: 61073.0000 - fn: 528.0000 - accuracy: 0.6749 - precision: 0.0369 - recall: 0.6810 - auc: 0.7307 - prc: 0.0676 - sensitivity: 0.7873 - specificity: 0.8248 - 21s/epoch - 15ms/step
(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805)  [repeated 7x across cluster]
(launch_and_evaluate 

(raylet) Spilled 4322 MiB, 1 objects, write throughput 178 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

DEBUG flwr 2023-06-13 17:38:38,863 | server.py:232 | fit_round 2 received 18 results and 0 failures
DEBUG:flwr:fit_round 2 received 18 results and 0 failures
DEBUG flwr 2023-06-13 17:38:38,888 | server.py:168 | evaluate_round 2: strategy sampled 19 clients (out of 24)
DEBUG:flwr:evaluate_round 2: strategy sampled 19 clients (out of 24)


(launch_and_evaluate pid=9811) 
(launch_and_evaluate pid=9811) This is client:  12
(launch_and_evaluate pid=9811) Loaded data for client:  12 
(launch_and_evaluate pid=9811) 
(launch_and_evaluate pid=9811) Making model:  12
(launch_and_fit pid=9811) 56/56 - 3s - loss: 0.6069 - tp: 671.0000 - fp: 342.0000 - tn: 548.0000 - fn: 219.0000 - accuracy: 0.6848 - precision: 0.6624 - recall: 0.7539 - auc: 0.7444 - prc: 0.7358 - sensitivity: 0.8146 - specificity: 0.8315 - val_loss: 0.5765 - val_tp: 622.0000 - val_fp: 270.0000 - val_tn: 502.0000 - val_fn: 150.0000 - val_accuracy: 0.7280 - val_precision: 0.6973 - val_recall: 0.8057 - val_auc: 0.7749 - val_prc: 0.7334 - val_sensitivity: 0.8977 - val_specificity: 0.8174 - 3s/epoch - 54ms/step [repeated 2x across cluster]
(launch_and_evaluate pid=9811) Client CID: 12 is done.
(launch_and_evaluate pid=9811) Hi, this is base station 12 in eval for round 2
(launch_and_evaluate pid=9811) 1440/1440 - 22s - loss: 0.6248 - tp: 1113.0000 - fp: 28063.0000 - tn

(raylet) Spilled 69160 MiB, 16 objects, write throughput 234 MiB/s.


(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805) Writing out interim results.
(launch_and_evaluate pid=9805)  [repeated 5x across cluster]
(launch_and_evaluate pid=9805) This is client:  2
(launch_and_evaluate pid=9805) Loaded data for client:  2 
(launch_and_evaluate pid=9805) Making model:  2
(launch_and_evaluate pid=9805) Client CID: 2 is done.
(launch_and_evaluate pid=9805) Hi, this is base station 2 in eval for round 2
(launch_and_evaluate pid=9811) 1440/1440 - 24s - loss: 0.6211 - tp: 1054.0000 - fp: 28022.0000 - tn: 62573.0000 - fn: 511.0000 - accuracy: 0.6904 - precision: 0.0362 - recall: 0.6735 - auc: 0.7445 - prc: 0.0655 - sensitivity: 0.8089 - specificity: 0.8360 - 24s/epoch - 16ms/step
(launch_and_evaluate pid=9811) Writing out interim results.
(launch_and_evaluate pid=9811)  [repeated 5x across cluster]
(launch_and_evaluate pid=9811) This is client:  11
(launch_and_evaluate pid=9811) Loaded data for client:  11 
(launch_and_evaluat

DEBUG flwr 2023-06-13 17:42:46,884 | server.py:182 | evaluate_round 2 received 19 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 19 results and 0 failures
INFO flwr 2023-06-13 17:42:46,887 | server.py:147 | FL finished in 1241.858615801
INFO:flwr:FL finished in 1241.858615801
INFO flwr 2023-06-13 17:42:46,892 | app.py:218 | app_fit: losses_distributed [(1, 0.6432136533688617), (2, 0.6231843320682983)]
INFO:flwr:app_fit: losses_distributed [(1, 0.6432136533688617), (2, 0.6231843320682983)]
INFO flwr 2023-06-13 17:42:46,897 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-13 17:42:46,899 | app.py:220 | app_fit: metrics_distributed {'tp': [(1, 1232.1650801021592), (2, 1237.5685005393743)], 'fp': [(1, 29689.124990325825), (2, 28161.97619561309)], 'tn': [(1, 60639.39989164926), (2, 62144.06695433297)], 'fn': [(1, 599.3100379227614), (2, 616.3883495145631)], 'accuracy': [(1, 0.6713494437092017), (2, 0.687734755994

In [34]:
# TODO: This cell needs turned into a function.
# Write out the results of evaluation.

count = 0
for key in (fl_history.metrics_distributed):
    print(key)
    results[count][1] = fl_history.metrics_distributed[key][(num_rounds-1):][0][1]
    count += 1
count = 0
results[11][1] = fl_epochs
results[12][1] = num_rounds
results[13][1] = NUM_CLIENTS * FRACTION_FIT[0]
results[14][1] = NUM_CLIENTS * FRACTION_EVALUATE[0]

file = results_path + "history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
results_out = pd.DataFrame(results)
results_out.columns = ['single_server', 'distributed']
results_out.index = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'sensitivity', 'specificity', 'epochs', 'rounds', 'train_clients', 'evaluate_clients']
print(results_out)
pd.DataFrame(results_out).to_csv(file)
print("\nWriting out results.")

tp
fp
tn
fn
accuracy
precision
recall
auc
prc
sensitivity
specificity
                  single_server   distributed
tp                   885.000000   1237.568501
fp                 11002.000000  28161.976196
tn                 33717.000000  62144.066954
fn                   476.000000    616.388350
accuracy               0.750911      0.687735
precision              0.074451      0.042065
recall                 0.650257      0.667432
auc                    0.763666      0.735389
prc                    0.087752      0.066190
sensitivity            0.861132      0.801345
specificity            0.843802      0.816174
epochs                10.000000      2.000000
rounds                 1.000000      2.000000
train_clients          1.000000     18.000000
evaluate_clients       1.000000     19.200000

Writing out results.


In [35]:
# TODO: Create a function that does all this
# Assemble all of the client round data into one file.
for filename in glob.glob(os.path.join(temp_path, '*.csv')):
    # print("\nnow reading " + filename + "\n")
    # read file
    df = pd.read_csv(filename, index_col=[0])
    client = int(df['0'].iloc[0])
    round = int(df['0'].iloc[1])

    # writing individual interim results to master interim results file
    interim_results[client:client+1, 0][:,round-1] = df['0'].iloc[2]
    interim_results[client:client+1, 1][:,round-1] = df['0'].iloc[3]
    interim_results[client:client+1, 2][:,round-1] = df['0'].iloc[4]
    interim_results[client:client+1, 3][:,round-1] = df['0'].iloc[5]
    interim_results[client:client+1, 4][:,round-1] = df['0'].iloc[6]
    interim_results[client:client+1, 5][:,round-1] = df['0'].iloc[7]
    interim_results[client:client+1, 6][:,round-1] = df['0'].iloc[8]
    interim_results[client:client+1, 7][:,round-1] = df['0'].iloc[9]
    interim_results[client:client+1, 8][:,round-1] = df['0'].iloc[10]
    interim_results[client:client+1, 9][:,round-1] = df['0'].iloc[11]
    interim_results[client:client+1, 10][:,round-1] = df['0'].iloc[12]

In [36]:

file = results_path + "interim_history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".npy"
# Write consolodated interim history to disk
np.save(file, interim_results)

In [37]:
# Read the array from the file
interim_results_test = np.load(file)
print(interim_results_test)

[[[ 1076.          1074.        ]
  [30412.         28374.        ]
  [60125.         62163.        ]
  [  547.           549.        ]
  [    0.66407335     0.68616539]
  [    0.03417175     0.03647107]
  [    0.66296983     0.6617375 ]
  [    0.72636509     0.73539478]
  [    0.06808017     0.0656634 ]
  [    0.7590881      0.77818853]
  [    0.84572053     0.83105248]]

 [[ 1052.             0.        ]
  [29837.             0.        ]
  [60745.             0.        ]
  [  526.             0.        ]
  [    0.67054039     0.        ]
  [    0.03405743     0.        ]
  [    0.66666669     0.        ]
  [    0.72871578     0.        ]
  [    0.0681777      0.        ]
  [    0.765526       0.        ]
  [    0.85210085     0.        ]]

 [[    0.          1054.        ]
  [    0.         28065.        ]
  [    0.         62537.        ]
  [    0.           504.        ]
  [    0.             0.69000649]
  [    0.             0.0361963 ]
  [    0.             0.67650837]
  [    0. 

In [38]:
# delete temp files
# https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
folder = temp_path
print("deleting files in: ", folder)
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

deleting files in:  /content/drive/MyDrive/Colab Notebooks/FF/history_temp/
